In [1]:
##########################################################################################################
# VERSION  	DESARROLLADOR             FECHA        DESCRIPCION
# -------------------------------------------------------------
#  1        Walter Albites Azarte     10/12/2021   Curso PySpark Entorno Local - Dataframe
##########################################################################################################

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.1.2-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf= pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [5]:
#Crear Dataframe Spark personas
schemaPersona = StructType([
    StructField("dni", StringType(),True),
    StructField("nombre", StringType(),True),
    StructField("edad", IntegerType(),True)
])

dataPersona = ([("99999999", "Walter",35),
                ("88888888", "Susan",30),
                ("77777777", "Alejandro",12)
               ])

df_personas=spark.createDataFrame(dataPersona,schema=schemaPersona)
df_personas.show()

#Crear Dataframe Spark Cursos
schemaCursos = StructType([
    StructField("dni", StringType(),True),
    StructField("curso", StringType(),True),
    StructField("precio", DoubleType(),True)
])

dataCurso = ([("99999999", "Spark",100.50),
              ("99999999", "Scala",100.50),
              ("99999999", "Java",100.50),
              ("88888888", "Ingles",80.90),
              ("77777777", "Java",12.50)
])

df_cursos=spark.createDataFrame(dataCurso,schema=schemaCursos)
df_cursos.show()


+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|88888888|    Susan|  30|
|77777777|Alejandro|  12|
+--------+---------+----+

+--------+------+------+
|     dni| curso|precio|
+--------+------+------+
|99999999| Spark| 100.5|
|99999999| Scala| 100.5|
|99999999|  Java| 100.5|
|88888888|Ingles|  80.9|
|77777777|  Java|  12.5|
+--------+------+------+



In [6]:
# Conteo de Registros
df_personas.count()

3

In [7]:
#Seleccionar campos
df_personasx=df_personas.select('dni','nombre')
df_personasx.show()

+--------+---------+
|     dni|   nombre|
+--------+---------+
|99999999|   Walter|
|88888888|    Susan|
|77777777|Alejandro|
+--------+---------+



In [8]:
#Adicionar columnas y agregar el valor Perú
df_personasx=df_personas.withColumn('pais',lit("Perú"))
df_personasx.show()

+--------+---------+----+----+
|     dni|   nombre|edad|pais|
+--------+---------+----+----+
|99999999|   Walter|  35|Perú|
|88888888|    Susan|  30|Perú|
|77777777|Alejandro|  12|Perú|
+--------+---------+----+----+



In [9]:
#Concatenar nombre y pais
df_personasx=df_personasx.withColumn('nombre_pais',concat(col("pais"),lit(" "),col("nombre")))
df_personasx.show()

+--------+---------+----+----+--------------+
|     dni|   nombre|edad|pais|   nombre_pais|
+--------+---------+----+----+--------------+
|99999999|   Walter|  35|Perú|   Perú Walter|
|88888888|    Susan|  30|Perú|    Perú Susan|
|77777777|Alejandro|  12|Perú|Perú Alejandro|
+--------+---------+----+----+--------------+



In [10]:
#Substring extraer los cuatro digitos de nombre_pais
df_personasx=df_personasx.withColumn('paisx',col("nombre_pais").substr(1,4))
df_personasx.show()


+--------+---------+----+----+--------------+-----+
|     dni|   nombre|edad|pais|   nombre_pais|paisx|
+--------+---------+----+----+--------------+-----+
|99999999|   Walter|  35|Perú|   Perú Walter| Perú|
|88888888|    Susan|  30|Perú|    Perú Susan| Perú|
|77777777|Alejandro|  12|Perú|Perú Alejandro| Perú|
+--------+---------+----+----+--------------+-----+



In [11]:
#Spark filter() o where() se usa para filtrar las filas de DataFrame o Dataset en función de una o varias condiciones

In [12]:
#Filtrar con una condicion
df_personasx=df_personas.filter(col("edad")>=30)
df_personasx.show()

+--------+------+----+
|     dni|nombre|edad|
+--------+------+----+
|99999999|Walter|  35|
|88888888| Susan|  30|
+--------+------+----+



In [13]:
#Filtrar con dos condiciones and 
df_personasx=df_personas.filter((col("edad")>=30) & (col("nombre")=="Walter"))
df_personasx.show()

+--------+------+----+
|     dni|nombre|edad|
+--------+------+----+
|99999999|Walter|  35|
+--------+------+----+



In [14]:
#Filtrar con dos condiciones or 
df_personasx=df_personas.filter((col("nombre")=="Alejandro") | (col("nombre")=="Walter"))
df_personasx.show()

+--------+---------+----+
|     dni|   nombre|edad|
+--------+---------+----+
|99999999|   Walter|  35|
|77777777|Alejandro|  12|
+--------+---------+----+



In [15]:
#Where con una condicion
df_personasx=df_personas.where(col("edad")>=30)
df_personasx.show()

+--------+------+----+
|     dni|nombre|edad|
+--------+------+----+
|99999999|Walter|  35|
|88888888| Susan|  30|
+--------+------+----+



In [16]:
df_personas.printSchema()

root
 |-- dni: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)



In [17]:
# Cambiar tipos de datos de Integer a String en una nueva columna
df_personasx = df_personas.withColumn("age",col("edad").cast(StringType()))

In [18]:
df_personasx.printSchema()
df_personasx.show()

root
 |-- dni: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- age: string (nullable = true)

+--------+---------+----+---+
|     dni|   nombre|edad|age|
+--------+---------+----+---+
|99999999|   Walter|  35| 35|
|88888888|    Susan|  30| 30|
|77777777|Alejandro|  12| 12|
+--------+---------+----+---+



In [19]:
# Agrupamiento contar registros
df_cursos.show()
df_cursos.groupBy("precio").count().show()

+--------+------+------+
|     dni| curso|precio|
+--------+------+------+
|99999999| Spark| 100.5|
|99999999| Scala| 100.5|
|99999999|  Java| 100.5|
|88888888|Ingles|  80.9|
|77777777|  Java|  12.5|
+--------+------+------+

+------+-----+
|precio|count|
+------+-----+
|  80.9|    1|
|  12.5|    1|
| 100.5|    3|
+------+-----+



In [20]:
# Agrupamiento sumar
df_cursos.groupBy("precio").sum().show()

+------+-----------+
|precio|sum(precio)|
+------+-----------+
|  80.9|       80.9|
|  12.5|       12.5|
| 100.5|      301.5|
+------+-----------+



In [21]:
df_cursos.groupBy("curso")\
.agg(sum("precio").alias("sum_precio"), \
         avg("precio").alias("avg_precio"), \
         max("precio").alias("max_precio") \
     ) \
    .show()

+------+----------+----------+----------+
| curso|sum_precio|avg_precio|max_precio|
+------+----------+----------+----------+
|Ingles|      80.9|      80.9|      80.9|
| Scala|     100.5|     100.5|     100.5|
| Spark|     100.5|     100.5|     100.5|
|  Java|     113.0|      56.5|     100.5|
+------+----------+----------+----------+



In [22]:
# Case when múltiples condiciones
df_cursosx=df_cursos.withColumn("categoria",when((col("curso")=="Scala") | (col("curso")=="Spark"),"Data")
                     .when(col("curso")=="Java","Back End")
                     .otherwise("Otro")
                    )

In [23]:
df_cursosx.show()

+--------+------+------+---------+
|     dni| curso|precio|categoria|
+--------+------+------+---------+
|99999999| Spark| 100.5|     Data|
|99999999| Scala| 100.5|     Data|
|99999999|  Java| 100.5| Back End|
|88888888|Ingles|  80.9|     Otro|
|77777777|  Java|  12.5| Back End|
+--------+------+------+---------+



In [24]:
#join personas cursos how='right', how='left',how='inner'
df_personas_with_df_cursos=df_personas.alias("a").join(df_cursos.alias("b"),col("a.dni")==col("b.dni"),how='inner')

In [25]:
df_personas_with_df_cursos.show()

+--------+---------+----+--------+------+------+
|     dni|   nombre|edad|     dni| curso|precio|
+--------+---------+----+--------+------+------+
|88888888|    Susan|  30|88888888|Ingles|  80.9|
|99999999|   Walter|  35|99999999| Spark| 100.5|
|99999999|   Walter|  35|99999999| Scala| 100.5|
|99999999|   Walter|  35|99999999|  Java| 100.5|
|77777777|Alejandro|  12|77777777|  Java|  12.5|
+--------+---------+----+--------+------+------+



In [26]:
#eliminar columna para que se pueda escribir y no muestre el error de  columnas duplicadas
df_personas_with_df_cursos=df_personas_with_df_cursos.drop(col("b.dni"))
df_personas_with_df_cursos.show()

+--------+---------+----+------+------+
|     dni|   nombre|edad| curso|precio|
+--------+---------+----+------+------+
|88888888|    Susan|  30|Ingles|  80.9|
|99999999|   Walter|  35| Spark| 100.5|
|99999999|   Walter|  35| Scala| 100.5|
|99999999|   Walter|  35|  Java| 100.5|
|77777777|Alejandro|  12|  Java|  12.5|
+--------+---------+----+------+------+



In [ ]:
# Escribir guardar dataframe 
df_personas_with_df_cursos.write.format("csv").mode("overwrite").save(r"C:\Users\walbi\Documents\personas_cursos.csv")

In [ ]:
# Union
df_personas.union(df_cursos).show()

In [ ]:
from datetime import date
from datetime import datetime
#Fecha actual
today = date.today()
print(today)
#Fecha actual y Hora
now = datetime.now()
print(now)
#Fecha actual solo año
print(today.year)

In [ ]:
# UDF funciones definidas por el usuario, caluclar el año de nacimiento segun la edad
udf_calcular_nacimiento=udf(lambda x: (today.year-x),IntegerType())
df_personasx=df_personas.withColumn("nacimiento",udf_calcular_nacimiento(col("edad")))

In [ ]:
df_personasx.show(10,False)

In [ ]:
#Crear Dataframe Spark Institutos
schemaPersonaInstituto = StructType([
    StructField("nif", StringType(),True),
    StructField("instituto", StringType(),True)
])

dataPersonaInstituto = ([
                ("30303030",'Institute of Experimental Physics##Faculty of Physics##University of Warsaw'),
                ("40404040",'Institute of Experimental Physics##Faculty of Physics##University of Warsaw##Perú'),
               ])

df_instituto=spark.createDataFrame(dataPersonaInstituto,schema=schemaPersonaInstituto)
df_instituto.show(5,False)

In [ ]:
#Split
df_instituto.select('instituto',
                    split(col('instituto'),'##').alias('lista'),
                    split(col('instituto'),'##')[0].alias('primerValor'),
                    split(col('instituto'),'##')[1].alias('SegundoValor'),
                    split(col('instituto'),'##')[2].alias('TercerValor')
                   ).show(5,False)

In [ ]:
#Split
df_instituto.select('instituto',
                   size(split(col('instituto'),'##')).alias('cant_nombre'),
                  split(col('instituto'),'##')[size(split(col('instituto'),'##'))-1].alias('a'),
                   split(col('instituto'),'##')[size(split(col('instituto'),'##'))-2].alias('b'),
                   split(col('instituto'),'##')[size(split(col('instituto'),'##'))-3].alias('c')
                  ).show(20,False)